In [1]:
# installs packages into kernel, not environment
import sys
!{sys.executable} -m pip install numpy geopandas pandas matplotlib momepy networkx shapely numpy


In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import momepy
import networkx as nx
import shapely
import numpy as np

import warnings
warnings.filterwarnings("ignore")

output_folder = "C:/Users/danwillett/Code/jiahua-connectivity/connectivity/scripts/python/outputs/"

c:\Users\danwillett\AppData\Local\anaconda3\envs\jiahua-connectivity\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
bike_network_path = '../places/Santa_barbara_county__California/bike_network_new.shp'
original_bike_network = gpd.read_file(bike_network_path)

low_stress_path = '../places/Santa_barbara_county__California/low_stress_network_new.shp'
original_low_stress = gpd.read_file(low_stress_path)
original_low_stress = original_low_stress.to_crs(original_bike_network.crs)

# all_streets_path = '../../data/all_street_pfb_SB/neighborhood_ways.shp'
# original_all_streets = gpd.read_file(all_streets_path)
# original_all_streets = original_all_streets.to_crs(original_bike_network.crs)

block_path = '../../data/20231120_blockgroup/var1120.shp'
blocks = gpd.read_file(block_path)
blocks = blocks.to_crs(original_bike_network.crs)

simp_net_path = '../../../places/Santa_barbara_county__California/network_new.shp'
simp_net = gpd.read_file(simp_net_path)
simp_net = simp_net.to_crs(original_bike_network.crs)


In [5]:
# find overlap between bike network and simplified network. Assign canbics classification to simplified network for edges with greatest overlap
def simplify_bike_network(orig_bike_network, simp_net, output_name):
    buffer_dist = 5 # meters based on crs
    simp_net['buffered_geometry'] = simp_net['geometry'].buffer(buffer_dist) # buffer so intersection will work
    orig_bike_network['saved_geom'] = orig_bike_network.geometry
    joined_data = gpd.sjoin(simp_net, orig_bike_network, how='left', op='intersects')
    joined_data = joined_data.dropna(subset=['saved_geom'])
    
    joined_data['overlap_area'] = joined_data.apply(lambda row: row['saved_geom'].buffer(.5).intersection(row['buffered_geometry']).area, axis=1)
    joined_data['left_index'] = joined_data['index']
    joined_data = joined_data.reset_index(drop=True)
    max_idx = joined_data.groupby('left_index')['overlap_area'].idxmax() # find index of highest overlap segment
    simplified_bike_network = joined_data.iloc[max_idx]
    
    simplified_bike_network.drop(columns=['buffered_geometry', 'saved_geom']).to_file(''.join([output_folder, output_name]), driver='ESRI Shapefile', if_exists='replace')
    print("simplified bike_network")
    return simplified_bike_network

# extract simplify networks with a low stress or all streets networks mask
def simplify_networks(network, simp_net, output_name):
    buffer_dist = 10 # meters
    clipped_simp = gpd.clip(simp_net, network['geometry'].buffer(buffer_dist)).explode()
    clipped_simp['new_length'] = clipped_simp['geometry'].length
    clipped_simp = clipped_simp[clipped_simp['new_length'] > 0.5 * clipped_simp['length']]
    
    clipped_simp.drop(columns="buffered_geometry").to_file(''.join([output_folder, output_name]), driver='ESRI Shapefile', if_exists='replace')
    print("masked network")
    return clipped_simp
    
# remove crosswalk paths from low_stress and all_streets
original_low_stress = original_low_stress[original_low_stress['XWALK'] == 0]
# original_all_streets = original_all_streets[original_all_streets['XWALK'] != 1]

# transfer bike_network classifications to simplified osm road network
bike_network = simplify_bike_network(original_bike_network, simp_net, "simplified_bike_network")
low_stress = simplify_networks(original_low_stress, simp_net, "simplified_low_stress")
# all_streets = simplify_networks(original_all_streets, simp_net, "simplified_all_streets")

simplified bike_network
masked network


In [8]:
# clean up roads in gpds

# remove multilinestrings from the low_stress dataframe. These are small roads that have been breaks between them, 
# but aren't located on primary roads
def remove_multis(shp_file, multi_output_name):
    geoms = shp_file['geometry']
    geomTypes = pd.Series([type(line) for line in geoms])
    multiTypes = geoms.apply(lambda x: isinstance(x, shapely.geometry.multilinestring.MultiLineString))
    multiLines = shp_file[multiTypes]
    
    if len(multiLines) > 0:
        # save multiline polygons to see how many are removed and if they are important
        output_shapefile_path = './outputs/' + multi_output_name
        multiLines.to_file(output_shapefile_path, driver='ESRI Shapefile')
    
    return shp_file[~multiTypes]

# bike_networks: only use roads with canbics classifications of 1-3
desired_comfort_levels = ['1. High Comfort', '2. Medium Comfort', '3. Low Comfort']
bike_network = bike_network[bike_network['Can_BICS_c'].isin(desired_comfort_levels)]

# remove polylines from gpds
bike_polylines = remove_multis(bike_network, 'bike_multilines')
low_stress_polylines = remove_multis(low_stress, 'low_stress_multilines')
all_streets_polylines = remove_multis(simp_net, 'all_streets_multlines')


In [9]:

# Calculating conductivity indicators
# network density = (total length of bike links)/(block area)
def calc_net_density(bike_edges, polygon):
    total_len = sum(bike_edges['edge_length'])/1000 # covert to km
    total_area = polygon['area'].values[0]/1000000 #convert to km2
    net_density = total_len/total_area # need to figure out what my data layers are
    return net_density

# Gamma Connectivity (degree of connectivity) = edges/lmax, where lmax = 3(n-2)
def calc_gamma(nodes, edges):
    # gamma connectivity only works if there are 3 or more nodes
    gamma_connectivity = (sum(edges['weights']))/(3*(len(nodes)-2)) if len(nodes) > 2 else None
    return gamma_connectivity

# Degree of network coverage (Number of bike links)/(number of street links)
def calc_net_coverage(bike_edges, street_edges):
    net_coverage = sum(bike_edges['weights'])/sum(street_edges['weights'])
    return net_coverage

# Intersection density (Number of bike network intersections)/(block area)
def calc_int_density(bike_nodes, polygon):
    total_area = polygon['area'].values[0]/1000000 #convert to km2
    int_density = len(bike_nodes)/total_area
    return int_density

# Degree of network complexity (Number of bike links)/(bike nodes)
def calc_complexity(bike_edges, bike_nodes):
    net_complexity = sum(bike_edges['weights'])/(len(bike_nodes))
    return net_complexity

In [11]:
print(bike_polylines)

       index     name_left     highway  group  is_simplif      length  \
0          1     unnamed_8     service   -1.0         0.0  241.961981   
12         7    unnamed_19     service   -1.0         0.0   43.374139   
16         8    unnamed_23     service   -1.0         0.0   79.778553   
18         9    unnamed_24     service   -1.0         0.0  172.666710   
272      164   unnamed_339     service   -1.0         0.0  490.581754   
...      ...           ...         ...    ...         ...         ...   
83562  65189  State Street  pedestrian    0.0         1.0   55.540430   
83565  65191  State Street  pedestrian    0.0         1.0   67.246820   
83576  65195  State Street  pedestrian    0.0         1.0   94.515426   
95258  65199  unnamed_7752     service   -1.0         0.0  241.610849   
95261  65200  unnamed_7751     service   -1.0         0.0  179.251699   

                                                geometry  \
0      LINESTRING (1831647.903 603279.182, 1831637.64...   
12 

In [10]:
# all units in meters (from crs)

# output census block shapefile path with appended connectivity indicator data
output_blocks_path = './outputs/connectivity_blocks'

# list to store connectivity indicators for each block
# b = bike_network, l = low_stress
connectivity_indicators_list = {
        'GEOID': [],
        'b_net_dens': [],
        'ls_net_dens': [],
        'b_gamma': [],
        'ls_gamma': [],
        'b_cover': [],
        'ls_cover': [],
        'b_int_dens': [],
        'ls_int_dens': [],
        'b_complex': [],
        'ls_complex': []
    }

# store edges and nodes to visualize in arcgis
bike_edges_list = []
bike_nodes_list = []
low_stress_edges_list = []
low_stress_nodes_list = []
street_edges_list = []
street_nodes_list = []

# iterate through each census block, calculating and storing connectivity indicators
for index, row in blocks.iterrows():
    row = row.to_frame().T # reshape row to be a dataframe with columns
    id = row['GEOID'].values[0] # store unique GEOID of the current census block
    polygon = gpd.GeoDataFrame(row, geometry='geometry', crs=blocks.crs)
    
    # select networks that reside in current block
    clipped_bike = gpd.sjoin(bike_polylines, polygon, how='inner', op='intersects')
    clipped_low_stress = gpd.sjoin(low_stress_polylines, polygon, how='inner', op='intersects')
    clipped_streets = gpd.sjoin(all_streets_polylines, polygon, how='inner', op='intersects')

    # find the new edge lengths of roads that are cut by block boundaries
    clipped_bike_edge = bike_polylines.clip(polygon)
    clipped_bike['edge_length'] = clipped_bike_edge['geometry'].length
    
    clipped_low_stress_edge = low_stress_polylines.clip(polygon)
    clipped_low_stress['edge_length'] = clipped_low_stress_edge['geometry'].length
    
    clipped_streets_edge = all_streets_polylines.clip(polygon)
    clipped_streets['edge_length'] = clipped_streets_edge['geometry'].length

    # calculate a weighting factor for clipped edges
    clipped_bike['weights'] = clipped_bike['edge_length']/clipped_bike['geometry'].length
    clipped_low_stress['weights'] = clipped_low_stress['edge_length']/clipped_low_stress['geometry'].length
    clipped_streets['weights'] = clipped_streets['edge_length']/clipped_streets['geometry'].length

    # Create a multigraph of nodes and edges where roads intersect
    bike_G = momepy.gdf_to_nx(clipped_bike, approach="primal")
    low_stress_G = momepy.gdf_to_nx(clipped_low_stress, approach="primal")
    street_G = momepy.gdf_to_nx(clipped_streets, approach="primal")
    
    # Getting nodes and edges
    
    if len(bike_G.nodes) > 0: # since the bike network is small, there are some polygons where there are no nodes or edges
        bike_nodes, bike_edges, bike_weights = momepy.nx_to_gdf(bike_G, spatial_weights=True)
    else:
        bike_nodes = None;
        bike_edges = None;

    low_stress_nodes, low_stress_edges, low_stress_weights = momepy.nx_to_gdf(low_stress_G, spatial_weights=True)
    street_nodes, street_edges, street_weights = momepy.nx_to_gdf(street_G, spatial_weights=True)

    # calculating connectivity indicators
    bike_network_density = calc_net_density(bike_edges, polygon) if bike_edges is not None else None
    low_stress_network_density = calc_net_density(low_stress_edges, polygon)

    bike_gamma = calc_gamma(street_nodes, bike_edges) if bike_edges is not None else None
    low_stress_gamma = calc_gamma(street_nodes, low_stress_edges)

    bike_network_coverage = calc_net_coverage(bike_edges, street_edges) if bike_edges is not None else None
    low_stress_network_coverage = calc_net_coverage(low_stress_edges, street_edges)

    bike_intersection_density = calc_int_density(bike_nodes, polygon) if bike_nodes is not None else None
    low_stress_intersection_density = calc_int_density(low_stress_nodes, polygon)
    
    bike_network_complexity = calc_complexity(bike_edges, bike_nodes) if bike_edges is not None else None
    low_stress_network_complexity = calc_complexity(low_stress_edges, low_stress_nodes)
    
    # store connectivity indicators in list
    connectivity_indicators_list['GEOID'].append(id)
    
    connectivity_indicators_list['b_net_dens'].append(bike_network_density)
    connectivity_indicators_list['ls_net_dens'].append(low_stress_network_density)
    
    connectivity_indicators_list['b_gamma'].append(bike_gamma)
    connectivity_indicators_list['ls_gamma'].append(low_stress_gamma)
    
    connectivity_indicators_list['b_cover'].append(bike_network_coverage)
    connectivity_indicators_list['ls_cover'].append(low_stress_network_coverage)

    connectivity_indicators_list['b_int_dens'].append(bike_intersection_density)
    connectivity_indicators_list['ls_int_dens'].append(low_stress_intersection_density)

    connectivity_indicators_list['b_complex'].append(bike_network_complexity)
    connectivity_indicators_list['ls_complex'].append(low_stress_network_complexity)

    # store edges and nodes
    if bike_edges is not None:
        bike_edges['GEOID'] = id
        bike_edges_list.append(bike_edges)
    if bike_nodes is not None:
        bike_nodes['GEOID'] = id
        bike_nodes_list.append(bike_nodes)
    low_stress_edges['GEOID'] = id
    low_stress_edges_list.append(low_stress_edges)
    low_stress_nodes['GEOID'] = id
    low_stress_nodes_list.append(low_stress_nodes)
    street_edges['GEOID'] = id
    street_edges_list.append(street_edges)
    street_nodes['GEOID'] = id
    street_nodes_list.append(street_nodes)
    
# Convert the list to a new DataFrame
connectivity_indicators_df = pd.DataFrame.from_dict(connectivity_indicators_list)
print(connectivity_indicators_df.head())

# Merge dataframe with blocks geodataframe on GEOID attribute
connectivity_blocks = blocks.merge(connectivity_indicators_df, on='GEOID')

# save updated census blocks to new file
connectivity_blocks.to_file(output_blocks_path, driver='ESRI Shapefile', if_exists='replace')

# convert edges + nodes to one gdf and save
nodes_list = [bike_nodes_list, low_stress_nodes_list, street_nodes_list]
nodes_name = ["bike_nodes", "low_stress_nodes", "street_nodes"]
nodes_file = './outputs/nodes/'
index = -1
for node in nodes_list:
    index += 1
    merged_nodes = gpd.GeoDataFrame(pd.concat(node, ignore_index=True), crs=node[0].crs)
    merged_nodes.to_file(nodes_file +nodes_name[index], driver='ESRI Shapefile', if_exists='replace')

edges_list = [bike_edges_list, low_stress_edges_list, street_edges_list]
edges_name = ["bike_edges", "low_stress_edges", "street_edges"]
edges_file = './outputs/edges/'
index = -1
for edge in edges_list:
    index += 1
    merged_edges = gpd.GeoDataFrame(pd.concat(edge, ignore_index=True), crs=edge[0].crs)
    merged_edges.to_file(edges_file + edges_name[index], driver='ESRI Shapefile', if_exists='replace')

ValueError: 'index_left' and 'index_right' cannot be names in the frames being joined

In [ ]:
# positions = {n: [n[0], n[1]] for n in list(G.nodes)}

# # Plot
# f, ax = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)
# low_stress_polylines.plot(color="k", ax=ax[0])
# for i, facet in enumerate(ax):
#     facet.set_title(("Streets", "Graph")[i])
#     facet.axis("off")
# nx.draw(G, positions, ax=ax[1], node_size=5)